In [ ]:
import sys
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
from scipy.ndimage import uniform_filter1d
import plotly.graph_objects as go
from typing import List, Tuple, Dict, Union
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.battery_model.battery_model import IRBatteryModel, ParallelGroup, BatteryPack

In [ ]:

# Create cells for the pack
cell = IRBatteryModel(3.9, 2000, 0.025)  # Example cell parameters
parallel_cells = 7  # 7p configuration
group = ParallelGroup(cell, parallel_cells)
groups_in_series = [group for _ in range(96)]  # 96s configuration

pack = BatteryPack(groups_in_series)

# Calculate and display pack characteristics at 2A current draw
current_draw = 2
print(f"Total Voltage: {pack.total_voltage():.3f}V")
print(f"Output Voltage at {current_draw}A: {pack.output_voltage_at_current(current_draw):.3f}V")
print(f"Power Loss at {current_draw}A: {pack.power_loss_at_current(current_draw):.3f}W")
print(f"Output Power at {current_draw}A: {pack.output_power_at_current(current_draw):.3f}W")
print(f"Total Internal Resistance: {pack.total_internal_resistance():.3f}Ω")
v, c, p = pack.get_pack_at_power(1000)
print(f"Voltage: {v:.3f}V, Current: {c:.3f}A, Power Loss: {p:.3f}W")


In [ ]:
v, c, p = pack.get_pack_at_power(30000)
print(f"Total Voltage: {pack.total_voltage():.3f}V, Voltage: {v:.3f}V, Current: {c:.3f}A, Power Loss: {p:.3f}W")

In [ ]:
# plot the voltage drop of the pack at different powers
powers = np.linspace(0, 90000, 100)
voltages = np.zeros(len(powers))
currents = np.zeros(len(powers))
power_losses = np.zeros(len(powers))
for i, power in enumerate(powers):
    v, c, p = pack.get_pack_at_power(power)
    voltages[i] = v
    currents[i] = c
    power_losses[i] = p

fig = px.line(x=powers, y=voltages, title="Voltage Drop of 96s7p Pack", labels={"x": "Power (W)", "y": "Voltage (V)"})
fig.show()

fig = px.line(x=powers, y=currents, title="Current Draw of 96s7p Pack", labels={"x": "Power (W)", "y": "Current (A)"})
fig.show()

fig = px.line(x=powers, y=power_losses, title="Power Loss of 96s7p Pack", labels={"x": "Power (W)", "y": "Power Loss (W)"})
fig.show()

In [ ]:
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car
from Functions.py_functions.gps_importer import *
from Functions.py_functions.lap_sims import RunSim
from Functions.py_functions.previous_cars import sr_9
from Functions.py_functions.tire_model.tire_model_utils import * # this has the tire models
from Functions.py_functions.las_solvers.octahedral import Octahedral_LAS
from Functions.py_functions.las_solvers.multi_layer import Multi_Layer_LAS
from Functions.py_functions.steady_state_solver.ls_optimize import LS_Solver
from Functions.py_functions.steady_state_solver.iterative import Iterative_Solver

In [ ]:
track = get_Crows_2023(50)
car = sr_9()
car.set_tire(H_R20_18X6_7)
# solver = LS_Solver()
solver = Iterative_Solver()
# las = Octahedral_LAS(solver=solver)
las = Multi_Layer_LAS(solver=solver)
# las.add_layers = 2
# car.debug = True
# set some settings
# car.debug = True
v_average = 15
mu = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.001
# make the simulation object
sim = RunSim(track, car, las)

In [ ]:
# run the simulation
sim.simulate(sim_type='qts', v_average=v_average, mu=mu, convergence_target=target, bins = 10) # This one is not fully functional yet, but here because it is easier to merge it before moving to github
# sim.simulate(sim_type='patton', v_average=v_average, mu=mu, convergence_target=target, bins = 10)
# sim.use_beta_init = True

In [ ]:
sim.plot_vs(distance=True, delta_beta_est=True, power_draw=True)

In [ ]:
# now we shall calculate the expected power lost as heat to the resistance in the battery
# we shall do this by calculating the power lost in the battery at each point in the lap
# and integrating based on u_time because the steps are currently in distance
power_draw = sim.motor_power
est_voltage, est_current, est_power_loss = np.zeros(len(power_draw)), np.zeros(len(power_draw)), np.zeros(len(power_draw))
for i in range(len(power_draw)):
    est_voltage[i], est_current[i], est_power_loss[i] = pack.get_pack_at_power(power_draw[i])

real_current, real_voltage = track.raw_track["MCM_DC_Bus_Current"]["Value"][0, 0][0, :], track.raw_track["MCM_DC_Bus_Voltage"]["Value"][0, 0][0, :]
dc_power = real_current * real_voltage
power_times = track.raw_track["MCM_DC_Bus_Current"]["Time"][0, 0][0, :] - track.start_time
power_dt = np.zeros(len(power_times))
power_dt[1:] = power_times[1:] - power_times[:-1]
dc_dist = np.interp(power_times, track.raw_time, track.interp_dist)
real_voltage, real_current, real_power_loss = np.zeros(len(dc_power)), np.zeros(len(dc_power)), np.zeros(len(dc_power))
for i in range(len(dc_power)):
    real_voltage[i], real_current[i], real_power_loss[i] = pack.get_pack_at_power(dc_power[i])

energy_use = np.cumsum(power_draw * sim.dt)
energy_draw = np.cumsum(est_power_loss * sim.dt)
energy_dc = np.cumsum(dc_power * power_dt)
energy_draw_dc = np.cumsum(real_power_loss * power_dt)

thermal_mass = 34560 # J/K
temp_rise_sim = energy_draw / thermal_mass
temp_rise_dc = energy_draw_dc / thermal_mass

temp_channels = []
for channel in track.raw_track.keys():
    if "_Temp" in channel and "BMS_Section_" in channel:
        temp_channels.append(channel)
        # print(track.raw_track[channel]["Value"][0, 0].shape)

# now we want to generate the average temprature of all of these channels
# We need to exclude any channels that are below 20C or above 75C because these are not valid
temp_time = track.raw_track[temp_channels[0]]["Time"][0, 0][0, :]
total, num = np.zeros(len(temp_time)), np.zeros(len(temp_time))
for i, channel in enumerate(temp_channels):
    temp = track.raw_track[channel]["Value"][0, 0][0, :]
    inds = np.where((temp > 20) & (temp < 75))
    total[inds] += temp[inds]
    num[inds] += 1
avg_temp = total / num

# for the sections where num is 0, we want to interpolate the temperature
inds = np.where(num == 0)
avg_temp[inds] = np.interp(temp_time[inds], temp_time[num != 0], avg_temp[num != 0])

# now we want to smooth the temperature
avg_temp = uniform_filter1d(avg_temp, size=200)

temp_dist = np.interp(temp_time - track.start_time, track.raw_time, track.interp_dist)

In [ ]:
# plot the voltage, current, power, and energy of the battery vs distance
fig = make_subplots(rows=5)
fig.add_trace(go.Scatter(x=track.u_crit, y=est_voltage, name="Estimated Voltage"), row=1, col=1)
fig.add_trace(go.Scatter(x=dc_dist, y=real_voltage, name="Real Voltage"), row=1, col=1)
fig.add_trace(go.Scatter(x=track.u_crit, y=est_current, name="Estimated Current"), row=2, col=1)
fig.add_trace(go.Scatter(x=dc_dist, y=real_current, name="Real Current"), row=2, col=1)
fig.add_trace(go.Scatter(x=track.u_crit, y=est_power_loss, name="Estimated Power Loss"), row=3, col=1)
fig.add_trace(go.Scatter(x=track.u_crit, y=sim.motor_power, name="Estimated Power Use"), row=3, col=1)
fig.add_trace(go.Scatter(x=dc_dist, y=dc_power, name="Real Power Use"), row=3, col=1)
fig.add_trace(go.Scatter(x=dc_dist, y=real_power_loss, name="Real Power Loss"), row=3, col=1)

fig.add_trace(go.Scatter(x=track.u_crit, y=energy_draw, name="Estimated Pack Heating"), row=4, col=1)
fig.add_trace(go.Scatter(x=track.u_crit, y=energy_use, name="Estimated Energy Use"), row=4, col=1)
fig.add_trace(go.Scatter(x=dc_dist, y=energy_dc, name="Real Energy Use"), row=4, col=1)
fig.add_trace(go.Scatter(x=dc_dist, y=energy_draw_dc, name="Real Energy Loss"), row=4, col=1)

fig.add_trace(go.Scatter(x=track.u_crit, y=temp_rise_sim + avg_temp[0], name="Sim Est Temp Rise"), row=5, col=1)
fig.add_trace(go.Scatter(x=dc_dist, y=temp_rise_dc + avg_temp[0], name="Data Est Temp Rise"), row=5, col=1)
fig.add_trace(go.Scatter(x=temp_dist, y=avg_temp, name="Real Temp"), row=5, col=1)
fig.update_xaxes(title_text="Distance (m)", row=5, col=1)
fig.update_yaxes(title_text="Voltage (V)", row=1, col=1)
fig.update_yaxes(title_text="Current (A)", row=2, col=1)
fig.update_yaxes(title_text="Power (W)", row=3, col=1)
fig.update_yaxes(title_text="Energy (J)", row=4, col=1)
fig.update_yaxes(title_text="Temperature (C)", row=5, col=1)
fig.update_layout(title="Battery Voltage, Current, Power, and Energy vs Distance", legend_title="Legend", height=1000)
fig.show()